In [1]:
pip install pymysql

Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 45.0/45.0 kB 551.3 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pymysql
import pandas as pd
import numpy

In [5]:
parse_dates = ['date_added']
netflix_titles = pd.read_csv('E:\DWH_Netflix_Project\Kaggle_Files\mycsvfile.csv', parse_dates=parse_dates)  
netflix_titles['date_added'] = netflix_titles['date_added'].dt.strftime("%Y-%m-%d") 
fullList = netflix_titles['title'].values.tolist()
genreList = list(set(netflix_titles['listed_in'].values.tolist()))

In [6]:
showDim = netflix_titles.drop(['date_added', 'release_year', 'rating', 'listed_in'], axis=1) 
showDim = showDim.where((pd.notnull(showDim)), None) 
showDimList = []
for row in showDim.values.tolist():
    showDimList.append(tuple(row))



In [7]:
netflix_origionals = pd.read_csv('E:/DWH_Netflix_Project/Kaggle_Files/netflix_originals.csv')

netflix_origionals = netflix_origionals[['Title','Seasons','Length','Netflix Exclusive Regions','Status']] 
netflix_origionals = netflix_origionals.where((pd.notnull(netflix_origionals)), None) 
netflix_origionals.drop_duplicates(subset ="Title", 
                     keep = 'first', inplace = True) 
netflix_origionalsList = [tuple(l) for l in netflix_origionals.values.tolist()] 


In [8]:
netflix_stocks = pd.read_csv('E:/DWH_Netflix_Project/Kaggle_Files/NFLX.csv', parse_dates=['Date'])
dateDim = pd.DataFrame({'date': pd.date_range(start='2002-05-23', end='2020-08-03')}) 
dateDim['date_id'] = dateDim.index + 1 
dateDim['date'] = dateDim['date'].dt.strftime("%Y-%m-%d") 
dateDim['year'] = pd.DatetimeIndex(dateDim['date']).year 
dateDim = dateDim.reindex(columns=['date_id','date','year']) 
dateDim.date_id = dateDim.date_id.astype(str)
dateDimList = [tuple(l) for l in dateDim.values.tolist()]

In [9]:
dateDim.date = pd.to_datetime(dateDim.date)

factsStockDF = pd.merge(netflix_stocks, dateDim, left_on='Date', right_on='date', how='inner')
factsStockDF = factsStockDF.drop(['Date', 'date', 'year'], axis=1) 
factsStockDF = factsStockDF.reindex(columns=['date_id','Open','High','Low','Close','Adj Close', 'Volume']) 
factsStockList = [tuple(l) for l in factsStockDF.values.tolist()] 

In [10]:
factsRating = pd.merge(netflix_titles, netflix_origionals, left_on='title', right_on='Title', how='left')  
factsRating.date_added = pd.to_datetime(factsRating.date_added) 

In [11]:
factsRatingDF = pd.merge(factsRating, dateDim, left_on='date_added', right_on='date', how='left')

In [12]:
factsRatingDF = factsRatingDF[['show_id','title','Title','date_id','listed_in','rating']]

In [13]:
factsRatingDF = factsRatingDF.where((pd.notnull(factsRatingDF)), None)

In [14]:
factsRatingList = [tuple(l) for l in factsRatingDF.values.tolist()]

In [38]:
# create show_dim table and insert data into it
db = pymysql.connect(
    host="localhost",
    user="root",
    password="123",
    database="Netflix_DB"
)
cursor = db.cursor()

cursor.execute("DROP TABLE IF EXISTS show_dim;") 


createTable1 = """CREATE TABLE show_dim(
                 show_id INT not null,
                 type VARCHAR(255) null,
                 title VARCHAR(255) not null,
                 director VARCHAR(255) null,
                 cast MEDIUMTEXT null,
                 country VARCHAR(255) null,
                 duration VARCHAR(255) null,
                 description MEDIUMTEXT null,
                 PRIMARY KEY (show_id, title));"""

cursor.execute(createTable1)

insertData1 = "INSERT INTO show_dim VALUES (%s,%s,%s,%s,%s,%s,%s,%s);"
cursor.executemany(insertData1, showDimList)
db.commit()

db.close() 

In [39]:
# create original_dim table and insert data into it
db = pymysql.connect(
    host="localhost",
    user="root",
    password="123",
    database="Netflix_DB"
)
cursor = db.cursor()

cursor.execute("DROP TABLE IF EXISTS original_dim;") 


createTable2 = """CREATE TABLE original_dim(
                  original_title VARCHAR(255) not null,
                  seasons VARCHAR(255) null,
                  length VARCHAR (255) null,
                  regions VARCHAR (255) null,
                  status VARCHAR (255) null,
                  PRIMARY KEY (original_title));"""

cursor.execute(createTable2)

insertData2 = "INSERT INTO original_dim VALUES (%s,%s,%s,%s,%s);"
cursor.executemany(insertData2, netflix_origionalsList)
db.commit()

db.close()

In [15]:
# create date_dim table and insert data into it
db = pymysql.connect(
    host="localhost",
    user="root",
    password="123",
    database="Netflix_DB"
)
cursor = db.cursor()
cursor.execute("DROP TABLE IF EXISTS date_dim;") 


createTable3 = """CREATE TABLE date_dim(
                  date_id VARCHAR(50) not null,
                  date DATE null,
                  year INT null,
                  PRIMARY KEY (date_id));"""

cursor.execute(createTable3)

insertData3 = "INSERT INTO date_dim VALUES (%s,%s,%s);"
cursor.executemany(insertData3, dateDimList)
db.commit()

db.close() 

In [43]:
# create genre_dim table and insert data into it
db = pymysql.connect(
    host="localhost",
    user="root",
    password="123",
    database="Netflix_DB"
)
cursor = db.cursor()

cursor.execute("DROP TABLE IF EXISTS genre_dim;") 


createTable4 = """CREATE TABLE genre_dim(
                  listed_in VARCHAR(255) not null,
                  PRIMARY KEY (listed_in));"""

cursor.execute(createTable4)

insertData4 = "INSERT INTO genre_dim VALUES (%s);" 
val = [[item] for item in genreList]
cursor.executemany(insertData4, val)
db.commit()

db.close()

In [16]:
# create facts_imdb_rating table and insert data into it
db = pymysql.connect(
    host="localhost",
    user="root",
    password="123",
    database="Netflix_DB"
)
cursor = db.cursor()

cursor.execute("DROP TABLE IF EXISTS facts_imdb_rating;") 


createTable5 = """CREATE TABLE facts_imdb_rating(
                  show_id INT not null,
                  title VARCHAR(255) not null,
                  original_title VARCHAR(255) null,
                  date_id VARCHAR(50) null,
                  listed_in VARCHAR(255) null,
                  rating FLOAT null,
                  PRIMARY KEY (show_id),
                  FOREIGN KEY (show_id, title) REFERENCES show_dim(show_id, title),
                  FOREIGN KEY (original_title) REFERENCES original_dim(original_title),
                  FOREIGN KEY (date_id) REFERENCES date_dim(date_id),
                  FOREIGN KEY (listed_in) REFERENCES genre_dim(listed_in)
                  );"""

cursor.execute(createTable5)

insertData5 = "INSERT INTO facts_imdb_rating VALUES (%s,%s,%s,%s,%s,%s);" 
cursor.executemany(insertData5, factsRatingList)
db.commit()

db.close() 

In [17]:
# create facts_stock_prices table and insert data into it
db = pymysql.connect(
    host="localhost",
    user="root",
    password="123",
    database="Netflix_DB"
)
cursor = db.cursor()

cursor.execute("DROP TABLE IF EXISTS facts_stock_prices;") 


createTable6 = """CREATE TABLE facts_stock_prices(
                  date_id VARCHAR(50) not null,
                  open FLOAT null,
                  high FLOAT null,
                  low FLOAT null,
                  close FLOAT null,
                  adj_close FLOAT null,
                  volume FLOAT null,
                  PRIMARY KEY (date_id),
                  FOREIGN KEY (date_id) REFERENCES date_dim(date_id)
                  );"""

cursor.execute(createTable6)

insertData6 = "INSERT INTO facts_stock_prices VALUES (%s,%s,%s,%s,%s,%s,%s);" 
cursor.executemany(insertData6, factsStockList)
db.commit()

db.close() 

In [18]:
import pymysql

db = pymysql.connect(
    host="localhost",
    user="root",
    password="123",
    database="Netflix_DB"
)

cursor = db.cursor()

try:
    
    cursor.execute("SELECT * FROM date_dim;")
    
    results = cursor.fetchall()

    for row in results:
        print(row)

except pymysql.Error as e:
    print(f"Error: {e}")

finally:
   
    db.close()


('1', datetime.date(2002, 5, 23), 2002)
('10', datetime.date(2002, 6, 1), 2002)
('100', datetime.date(2002, 8, 30), 2002)
('1000', datetime.date(2005, 2, 15), 2005)
('1001', datetime.date(2005, 2, 16), 2005)
('1002', datetime.date(2005, 2, 17), 2005)
('1003', datetime.date(2005, 2, 18), 2005)
('1004', datetime.date(2005, 2, 19), 2005)
('1005', datetime.date(2005, 2, 20), 2005)
('1006', datetime.date(2005, 2, 21), 2005)
('1007', datetime.date(2005, 2, 22), 2005)
('1008', datetime.date(2005, 2, 23), 2005)
('1009', datetime.date(2005, 2, 24), 2005)
('101', datetime.date(2002, 8, 31), 2002)
('1010', datetime.date(2005, 2, 25), 2005)
('1011', datetime.date(2005, 2, 26), 2005)
('1012', datetime.date(2005, 2, 27), 2005)
('1013', datetime.date(2005, 2, 28), 2005)
('1014', datetime.date(2005, 3, 1), 2005)
('1015', datetime.date(2005, 3, 2), 2005)
('1016', datetime.date(2005, 3, 3), 2005)
('1017', datetime.date(2005, 3, 4), 2005)
('1018', datetime.date(2005, 3, 5), 2005)
('1019', datetime.date(2

In [19]:
import pymysql

db = pymysql.connect(
    host="localhost",
    user="root",
    password="123",
    database="Netflix_DB"
)

cursor = db.cursor()

try:
    
    cursor.execute("SELECT * FROM facts_stock_prices;")
    
    results = cursor.fetchall()

    for row in results:
        print(row)

except pymysql.Error as e:
    print(f"Error: {e}")

finally:
   
    db.close()


('1', 1.15643, 1.24286, 1.14571, 1.19643, 1.19643, 104790000.0)
('100', 0.914286, 0.962857, 0.894286, 0.936429, 0.936429, 2121000.0)
('1000', 1.60714, 1.64286, 1.58429, 1.60714, 1.60714, 7910000.0)
('1001', 1.59571, 1.60571, 1.55143, 1.56429, 1.56429, 5604900.0)
('1002', 1.58286, 1.58286, 1.49143, 1.5, 1.5, 8947400.0)
('1003', 1.5, 1.53571, 1.49, 1.52571, 1.52571, 9433200.0)
('1007', 1.51429, 1.53571, 1.45, 1.45857, 1.45857, 6493900.0)
('1008', 1.46714, 1.48286, 1.44429, 1.45571, 1.45571, 4457600.0)
('1009', 1.45, 1.49571, 1.43, 1.48571, 1.48571, 5311600.0)
('1010', 1.52714, 1.58286, 1.52143, 1.57, 1.57, 29469300.0)
('1013', 1.58286, 1.58571, 1.5, 1.52857, 1.52857, 6608000.0)
('1014', 1.52857, 1.56429, 1.51571, 1.54143, 1.54143, 4452000.0)
('1015', 1.54286, 1.57857, 1.51429, 1.52571, 1.52571, 14411600.0)
('1016', 1.52714, 1.55571, 1.5, 1.50857, 1.50857, 7331800.0)
('1017', 1.51429, 1.54429, 1.5, 1.52857, 1.52857, 5499900.0)
('1020', 1.52714, 1.54571, 1.5, 1.51286, 1.51286, 4238500.0)
(

In [20]:
import pymysql

db = pymysql.connect(
    host="localhost",
    user="root",
    password="123",
    database="Netflix_DB"
)

cursor = db.cursor()

try:
    
    cursor.execute("SELECT * FROM facts_imdb_rating;")
    
    results = cursor.fetchall()

    for row in results:
        print(row)

except pymysql.Error as e:
    print(f"Error: {e}")

finally:
   
    db.close()

(247747, 'Amar Akbar Anthony', None, '6432', 'Action & Adventure, Classic Movies, Comedies', 7.6)
(269880, 'Bad Boys', None, '6341', 'Action & Adventure, Comedies', 7.0)
(281550, 'La Bamba', None, '6433', 'Classic Movies, Dramas, Music & Musicals', 7.1)
(284890, 'Barsaat', None, '5793', 'Action & Adventure, International Movies, Music & Musicals', 5.8)
(292118, 'Beavis and Butt-head Do America', None, '6391', 'Comedies, Cult Movies', 6.9)
(296682, 'Benji', 'Benji', '5767', 'Children & Family Movies, Classic Movies', 6.7)
(347365, 'Candyman', None, '6341', 'Cult Movies, Horror Movies', 6.7)
(352989, 'Carrie', None, '6219', 'Classic Movies, Cult Movies, Horror Movies', 7.4)
(372195, "Child's Play", None, '6432', 'Cult Movies, Horror Movies', 5.9)
(374651, 'Chitty Chitty Bang Bang', None, '6433', 'Children & Family Movies, Classic Movies, Comedies', 7.1)
(382650, 'Cleopatra Jones', None, '6372', 'Action & Adventure, Classic Movies', 6.0)
(404067, 'The Craft', None, '6433', 'Horror Movies,

In [21]:
import pymysql

db = pymysql.connect(
    host="localhost",
    user="root",
    password="123",
    database="Netflix_DB"
)

cursor = db.cursor()

try:
    
    cursor.execute("SELECT * FROM show_dim;")
    
    results = cursor.fetchall()

    for row in results:
        print(row)

except pymysql.Error as e:
    print(f"Error: {e}")

finally:
   
    db.close()

(247747, 'Movie', 'Amar Akbar Anthony', 'Manmohan Desai', 'Vinod Khanna, Rishi Kapoor, Amitabh Bachchan, Neetu Singh, Parveen Babi, Shabana Azmi, Nirupa Roy, Pran', 'India', '172 min', "Abandoned in a park by their father, Amar, Akbar and Anthony grow up independently and have no knowledge of one another. They're reunited as adults when, by coincidence, they all give blood at the same hospital.")
(269880, 'Movie', 'Bad Boys', 'Michael Bay', 'Will Smith, Martin Lawrence, Téa Leoni, Tchéky Karyo, Theresa Randle, Joe Pantoliano, Emmanuel Xuereb, Michael Imperioli, Nestor Serrano, Julio Oscar Mechoso', 'United States', '119 min', 'In this fast-paced actioner, two Miami narcotics cops are hot on the trail of a master crook who has snatched $100 million worth of heroin.')
(281550, 'Movie', 'La Bamba', 'Luis Valdez', 'Lou Diamond Phillips, Esai Morales, Rosanna DeSoto, Elizabeth Peña, Danielle von Zerneck, Joe Pantoliano, Rick Dees, Marshall Crenshaw, Howard Huntsberry, Brian Setzer', 'United

In [22]:
import pymysql

db = pymysql.connect(
    host="localhost",
    user="root",
    password="123",
    database="Netflix_DB"
)

cursor = db.cursor()

try:
    
    cursor.execute("SELECT * FROM original_dim;")
    
    results = cursor.fetchall()

    for row in results:
        print(row)

except pymysql.Error as e:
    print(f"Error: {e}")

finally:
   
    db.close()

('#blackAF', '1 season, 8 episodes', '32–48 min.', 'Worldwide', 'Renewed')
('#REALITYHIGH', None, '1 hour, 39 min.', 'Worldwide', None)
('100 Humans', '1 season, 8 episodes', '32-41 min.', 'Worldwide', 'Pending\n')
('13 Reasons Why', '4 seasons, 49 episodes', '49–98 min.', 'Worldwide', 'Ended')
('13 Reasons Why: Beyond the Reasons', '3 seasons, 3 episodes', '17–67 min.', 'Worldwide', 'Ended\n')
('13th', None, '1 hour, 40 min.', 'Worldwide', None)
('13th: A Conversation with Oprah Winfrey & Ava DuVernay', None, '36 min.', 'Worldwide', None)
('15-Aug', None, '2 hours, 4 min.', 'Worldwide', None)
('1922', None, '1 hour, 42 min.', 'Worldwide', None)
('1994', '5 episodes', '44–52 min.', 'Worldwide', 'Miniseries\n')
('22-Jul', None, '2 hours, 24 min.', 'Worldwide', None)
('3%', '3 seasons, 26 episodes', '38–51 min.', 'Worldwide', 'Renewed for final season')
('37 Seconds', None, '1 hour, 55 min.', 'Selected territories', None)
('4L', None, '1 hour, 44 min.', 'Selected territories', None)
('5 

In [23]:
import pymysql

db = pymysql.connect(
    host="localhost",
    user="root",
    password="123",
    database="Netflix_DB"
)

cursor = db.cursor()

try:
    
    cursor.execute("SELECT * FROM genre_dim;")
    
    results = cursor.fetchall()

    for row in results:
        print(row)

except pymysql.Error as e:
    print(f"Error: {e}")

finally:
   
    db.close()

('Action & Adventure',)
('Action & Adventure, Anime Features, Children & Family Movies',)
('Action & Adventure, Anime Features, Classic Movies',)
('Action & Adventure, Anime Features, Horror Movies',)
('Action & Adventure, Anime Features, International Movies',)
('Action & Adventure, Anime Features, Sci-Fi & Fantasy',)
('Action & Adventure, Children & Family Movies',)
('Action & Adventure, Children & Family Movies, Classic Movies',)
('Action & Adventure, Children & Family Movies, Comedies',)
('Action & Adventure, Children & Family Movies, Dramas',)
('Action & Adventure, Children & Family Movies, Independent Movies',)
('Action & Adventure, Children & Family Movies, Sci-Fi & Fantasy',)
('Action & Adventure, Classic Movies',)
('Action & Adventure, Classic Movies, Comedies',)
('Action & Adventure, Classic Movies, Cult Movies',)
('Action & Adventure, Classic Movies, Dramas',)
('Action & Adventure, Classic Movies, International Movies',)
('Action & Adventure, Classic Movies, Sci-Fi & Fantasy